In [2]:
https://ieeexplore.ieee.org/document/8870399

import numpy as np
import math

def levy_flight(Lambda=1.5, size=None):
    """Generate step sizes from a Lévy distribution (Mantegna’s method)"""
    sigma_u = ((math.gamma(1 + Lambda) * math.sin(math.pi * Lambda / 2)) /
               (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    sigma_v = 1.0
    u = np.random.normal(0, sigma_u, size=size)
    v = np.random.normal(0, sigma_v, size=size)
    step = u / (np.abs(v) ** (1.0 / Lambda))
    return step


def simple_cuckoo_search(fitness_fn, bounds, 
                         n_nests=25, pa=0.25, alpha=0.01, 
                         max_iter=200):
    """
    fitness_fn : function mapping x -> cost (lower is better)
    bounds : list of (lower, upper) for each dimension
    n_nests : number of nests (population)
    pa : probability of discovery (fraction of nests to replace)
    alpha : scaling factor for step size
    max_iter : max number of iterations
    """
    dim = len(bounds)
    # Initialize nests randomly
    nests = np.zeros((n_nests, dim))
    fitness = np.zeros(n_nests)
    for i in range(n_nests):
        nests[i] = np.array([random.uniform(bounds[j][0], bounds[j][1]) for j in range(dim)])
        fitness[i] = fitness_fn(nests[i])
    # Keep track of best
    best_idx = np.argmin(fitness)
    best_nest = nests[best_idx].copy()
    best_fit = fitness[best_idx]
    
    for t in range(max_iter):
        # For each nest, generate a new solution via levy flight
        for i in range(n_nests):
            # Lévy step
            step = levy_flight(size=dim)
            x_new = nests[i] + alpha * step * (nests[i] - best_nest)
            # enforce bounds
            for j in range(dim):
                x_new[j] = np.clip(x_new[j], bounds[j][0], bounds[j][1])
            f_new = fitness_fn(x_new)
            # choose a random nest to compare
            j = random.randrange(n_nests)
            if f_new < fitness[j]:
                nests[j] = x_new
                fitness[j] = f_new
        # Abandon a fraction pa of worst nests, replace by new ones
        n_abandon = int(pa * n_nests)
        # sort by fitness descending (worst first)
        worst_idxs = np.argsort(fitness)[-n_abandon:]
        for idx in worst_idxs:
            # new random nest
            nests[idx] = np.array([random.uniform(bounds[j][0], bounds[j][1]) for j in range(dim)])
            fitness[idx] = fitness_fn(nests[idx])
        # Update best
        cur_idx = np.argmin(fitness)
        if fitness[cur_idx] < best_fit:
            best_fit = fitness[cur_idx]
            best_nest = nests[cur_idx].copy()
        # (Optional: print progress)
        # print(f"Iter {t}: best = {best_fit}")
    return best_nest, best_fit

# Example: define fitness for elliptical antenna array
def antenna_fitness(x):
    """
    x: decision vector. Let's assume a simple model:
    First N entries: angular positions θ_i in [0, 2π]
    Next N entries: amplitudes a_i in [0,1]
    Next N entries: phases φ_i in [0, 2π]
    We compute the array factor on a grid of θ, evaluate some penalty.
    """
    N = len(x) // 3
    thetas = x[:N]
    amps = x[N:2*N]
    phs = x[2*N:3*N]
    # sample observation angles
    obs = np.linspace(0, np.pi, 181)  # e.g. 0 to 180 deg
    AF = np.zeros_like(obs, dtype=complex)
    # elliptical positions: assume ellipse a and b axes (hardcode or as extra)
    a = 1.0
    b = 0.5
    for i in range(N):
        theta_i = thetas[i]
        xi = a * np.cos(theta_i)
        yi = b * np.sin(theta_i)
        # compute steering vector contribution (simplified 2D)
        for k, th in enumerate(obs):
            # wave number k = 2π/λ; assume λ = 1
            k0 = 2 * np.pi
            r_dot = xi * np.sin(th)  + yi * 0  # simple projection
            AF[k] += amps[i] * np.exp(1j * (k0 * r_dot + phs[i]))
    AF_mag = np.abs(AF)
    # Normalize
    AF_db = 20 * np.log10(AF_mag / np.max(AF_mag) + 1e-12)
    # Compute side-lobe penalty: for θ outside main beam region (e.g. ±10°),
    # penalize any value above, say, –20 dB
    penalty = 0.0
    main_bw = 10 * np.pi / 180  # ±10 deg
    for k, th in enumerate(obs):
        if abs(th - np.pi/2) > main_bw:  # assume main beam near broadside
            if AF_db[k] > -20:
                penalty += (AF_db[k] + 20) ** 2
    # also include ripple in main lobe region
    # ...
    return penalty

if __name__ == "__main__":
    N_elem = 8
    dim = 3 * N_elem
    bounds = []
    for i in range(N_elem):  # θ_i
        bounds.append((0, 2 * np.pi))
    for i in range(N_elem):  # amplitude
        bounds.append((0.0, 1.0))
    for i in range(N_elem):  # phase
        bounds.append((0, 2 * np.pi))
    best_x, best_val = simple_cuckoo_search(antenna_fitness, bounds, 
                                            n_nests=30, pa=0.25, 
                                            alpha=0.05, max_iter=200)
    print("Best solution:", best_x)
    print("Best fitness:", best_val)


Best solution: [4.25038956 4.93454333 0.75719885 2.91383988 1.40342501 2.51461705
 2.91099354 5.43184488 0.70685309 0.97435012 0.21506567 0.0108814
 0.62804072 0.84062938 1.         0.33584374 3.42973747 3.98393202
 5.45640333 0.18229234 1.43679597 2.93129475 1.3361292  3.2731471 ]
Best fitness: 11244.243032603705
